In [1]:
from pyspark.sql.functions import *
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("Product_Price_Tracking") \
    .config("spark.jars.packages", "io.delta:delta-core_2.11:0.6.1,org.apache.hadoop:hadoop-aws:2.7.3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

from delta.tables import *

In [2]:
import configparser
import os
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
aws_section = 'default'

_AWS_KEY_ID = config.get(aws_section, "aws_access_key_id")
_AWS_SECRET_KEY = config.get(aws_section, "aws_secret_access_key")

spark.sparkContext._conf.setAll([('spark.delta.logStore.class','org.apache.spark.sql.delta.storage.S3SingleDriverLogStore')])
spark._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", _AWS_KEY_ID)
spark._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey", _AWS_SECRET_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")

In [3]:
df_productsaug20 = spark.read.csv('s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/raw/products_aug20.csv', header=True, inferSchema=True)
df_productsaug20.show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 20.5|
|      210|2020-08-20 00:00:00| 45.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|89.76|
+---------+-------------------+-----+



In [4]:
df_productsaug20.write.format("delta").option("path", "s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products").saveAsTable("products")

In [5]:
spark.sql("SELECT * FROM products").show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 20.5|
|      210|2020-08-20 00:00:00| 45.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|89.76|
+---------+-------------------+-----+



In [6]:
deltaTable = DeltaTable.forPath(spark, "s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products")

In [7]:
deltaTable.update("ProductID = '200'", { "Price": "'48.00'" } )

In [8]:
df = spark.read.format("delta").option("versionAsOf", 1).load("s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products")
df.show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 48.0|
|      210|2020-08-20 00:00:00| 45.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|89.76|
+---------+-------------------+-----+



In [9]:
df = spark.read.format("delta").option("versionAsOf", 0).load("s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products")
df.show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 20.5|
|      210|2020-08-20 00:00:00| 45.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|89.76|
+---------+-------------------+-----+



In [10]:
deltaTable.delete("ProductID = 210") 
df = spark.read.format("delta").option("versionAsOf", 2).load("s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products")
df.show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 48.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|89.76|
+---------+-------------------+-----+



In [11]:
df_productsaug21 = spark.read.csv('s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/raw/products_aug21.csv', header=True, inferSchema=True)
df_productsaug21.show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-21 00:00:00| 25.5|
|      210|2020-08-21 00:00:00| 46.0|
|      220|2020-08-21 00:00:00|34.56|
|      230|2020-08-21 00:00:00|23.67|
|      240|2020-08-21 00:00:00|90.82|
|      250|2020-08-21 00:00:00|89.76|
+---------+-------------------+-----+



In [12]:
deltaTable.alias("products").merge(
    df_productsaug21.alias("products_new"),
                    "products.ProductID = products_new.ProductID") \
                    .whenMatchedUpdate(set = { "Price" : "products_new.Price" } ) \
                    .whenNotMatchedInsert(values =
                       {
                        "ProductID": "products_new.ProductID",
                        "Date": "products_new.Date",
                        "Price": "products_new.Price"
                       }
                     ).execute()

In [13]:
df = spark.read.format("delta").option("versionAsOf", 3).load("s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products")
df.sort('ProductID').show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 25.5|
|      210|2020-08-21 00:00:00| 46.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|90.82|
|      250|2020-08-21 00:00:00|89.76|
+---------+-------------------+-----+



In [14]:
spark.table("products").sort('ProductID').show()

+---------+-------------------+-----+
|ProductID|               Date|Price|
+---------+-------------------+-----+
|      200|2020-08-20 00:00:00| 25.5|
|      210|2020-08-21 00:00:00| 46.0|
|      220|2020-08-20 00:00:00|34.56|
|      230|2020-08-20 00:00:00|23.67|
|      240|2020-08-20 00:00:00|90.82|
|      250|2020-08-21 00:00:00|89.76|
+---------+-------------------+-----+



In [15]:
deltaTable.update("ProductID = '200'", { "Price": "'35.5'" } )
deltaTable.update("ProductID = '210'", { "Price": "'56.00'" } )
deltaTable.update("ProductID = '220'", { "Price": "'44.56'" } )
deltaTable.update("ProductID = '230'", { "Price": "'33.67'" } )
deltaTable.update("ProductID = '240'", { "Price": "'100.82'" } )
deltaTable.update("ProductID = '250'", { "Price": "'99.76'" } )
deltaTable.update("ProductID = '260'", { "Price": "'64.55'" } )
deltaTable.update("ProductID = '270'", { "Price": "'106.32'" } )
deltaTable.update("ProductID = '280'", { "Price": "'54.78'" } )

In [16]:
s3_products_path = "s3a://odc-raw-data-ut-bucket/Arthur/delta_lake/products"
df = spark.read.format("delta").load(s3_products_path)

# df.show(df.count(), False)
df.count()
# df.filter(df.ProductID=="1").show()


6